# Drowsiness Detection OpenCV


This code can detect your eyes and alert when the user is drowsy.


In [1]:
#based on https://www.pyimagesearch.com/2017/05/08/drowsiness-detection-opencv/
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import cv2

In [2]:
def eye_aspect_ratio(eye): #measure when eyelids are closed
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear

In [3]:
thresh = 0.25
frame_check = 20
detect = dlib.get_frontal_face_detector() #create a face detector
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") #get data that contain faces

In [4]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"] #get array values that are only of the left eye
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"] #get array values that are only of the right eye

In [5]:
cap=cv2.VideoCapture(0)#start video
flag=0
while True:
    ret, frame=cap.read() #read frame
    frame = imutils.resize(frame, width=450) #reshape frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # make frame grayscale
    subjects = detect(gray, 0) # detect faces in gray image
    for subject in subjects:
        shape = predict(gray, subject) #predict face features based on faces in gray image
        shape = face_utils.shape_to_np(shape)#converting to NumPy Array
        leftEye = shape[lStart:lEnd] #left eye array points
        rightEye = shape[rStart:rEnd] #right eye array points
        leftEAR = eye_aspect_ratio(leftEye) # eye aspect ratio of left eye
        rightEAR = eye_aspect_ratio(rightEye) # eye aspect ratio of right eye
        ear = (leftEAR + rightEAR) / 2.0 #use average of both aspect ratios -> better estimation
        leftEyeHull = cv2.convexHull(leftEye) #get convex hull of left eye (a circle connecting all outer points of array)
        rightEyeHull = cv2.convexHull(rightEye) #get convex hull of right eye
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1) #draw based on convex hulls
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        # if eye aspect ratio is lower than threshold (eyes are drowsy) increase flag
        if ear < thresh:
            flag += 1
            #print (flag)
            if flag >= frame_check: #if eyes have been closed for too long -> alert
                cv2.putText(frame, "****************ALERT!****************", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "****************ALERT!****************", (10,325),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            flag = 0
    cv2.imshow("Frame", frame) #show frame
    key = cv2.waitKey(1) & 0xFF #get pressed key
    if key == ord("q"): #close if key is q
        cv2.destroyAllWindows()
        cap.release()
        break
